In [1]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 120 kB 65.9 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 


In [ ]:
!pip install sewar

In [12]:
!git clone https://github.com/pythonlessons/TensorFlow-2.x-YOLOv3

Cloning into 'TensorFlow-2.x-YOLOv3'...
remote: Enumerating objects: 800, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 800 (delta 1), reused 5 (delta 1), pack-reused 791
Receiving objects: 100% (800/800), 70.79 MiB | 20.05 MiB/s, done.
Resolving deltas: 100% (468/468), done.
Checking out files: 100% (57/57), done.


## Загрузим необоходимые библиотеки

In [2]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import datetime

In [13]:
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from deep_sort import generate_detections as gdet

In [ ]:
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/798783/train_dataset_train.zip

--2022-09-06 10:33:41--  https://lodmedia.hb.bizmrg.com/case_files/798783/train_dataset_train.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 894370794 (853M) [application/zip]
Saving to: ‘train_dataset_train.zip’

train_dataset_train 100%[===================>] 852.94M  10.9MB/s    in 80s     

2022-09-06 10:35:02 (10.7 MB/s) - ‘train_dataset_train.zip’ saved [894370794/894370794]



In [3]:
!wget https://lodmedia.hb.bizmrg.com/case_files/798783/test_dataset_test.zip

--2022-09-07 03:45:36--  https://lodmedia.hb.bizmrg.com/case_files/798783/test_dataset_test.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 217.69.142.100
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 333728618 (318M) [application/zip]
Saving to: ‘test_dataset_test.zip’

test_dataset_test.z 100%[===================>] 318.27M  11.5MB/s    in 30s     

2022-09-07 03:46:08 (10.6 MB/s) - ‘test_dataset_test.zip’ saved [333728618/333728618]



In [ ]:
!unzip train_dataset_train.zip

Archive:  train_dataset_train.zip
   creating: train/
  inflating: train/train.csv         
  inflating: train/train1.avi        
  inflating: train/train2.avi        


In [4]:
!unzip test_dataset_test.zip

Archive:  test_dataset_test.zip
   creating: test/
  inflating: test/test.avi           
  inflating: test/test.csv           


## Extract capture

In [ ]:
!rm -rf frames

In [ ]:
!mkdir frames

In [ ]:
!mkdir framas/test

mkdir: cannot create directory ‘framas/test’: No such file or directory


In [ ]:
VIDEO_DIR = '/content/test'
FRAMES_DIR = '/content/frames'

In [ ]:
start_date = datetime.datetime(2022, 5, 22, 8, 25, 47)
date1 = datetime.datetime(2022, 5, 22, 11, 51, 24)
diff = date1 - start_date 
print('Difference: ', diff)
  
print(diff.total_seconds())

Difference:  3:25:37
12337.0


In [ ]:
video_path = '/content/test/'
video_name = 'test.avi'

In [ ]:
video_path = '/content/train/'
video_name = 'train2.avi'

In [ ]:
def check_frame_image(img1, img2):
  diff = uqi(img1, img2)
  return diff < 0.9

In [ ]:
def check_frame_simularity(current_frame, prev_frame):
  diff_idx = []
  for i in range(len(current_frame)):
    if check_frame_image(current_frame[i], prev_frame[i]):
      diff_idx.append(i)
  return diff_idx

In [ ]:
prev_frame = None
current_frame = None

In [ ]:
cap = cv2.VideoCapture(os.path.join(video_path, video_name))

#Set frame_no in range 0.0-1.0
#In this example we have a video of 30 seconds having 25 frames per seconds, thus we have 750 frames.
#The examined frame must get a value from 0 to 749.
#For more info about the video flags see here: https://stackoverflow.com/questions/11420748/setting-camera-parameters-in-opencv-python
#Here we select the last frame as frame sequence=749. In case you want to select other frame change value 749.
#BE CAREFUL! Each video has different time length and frame rate. 
#So make sure that you have the right parameters for the right video!
# time_length = 30.0
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(fps)
print(frame_count)
seconds = frame_count / fps
minutes = int(seconds / 60)
print(minutes, seconds)
frame_seq = 19
frame_no = (frame_seq / (seconds*fps*100))
print(frame_no)

#The first argument of cap.set(), number 2 defines that parameter for setting the frame selection.
#Number 2 defines flag CV_CAP_PROP_POS_FRAMES which is a 0-based index of the frame to be decoded/captured next.
#The second argument defines the frame number in range 0.0-1.0
# cap.set(2, frame_no);
cap.set(cv2.CAP_PROP_POS_MSEC, 2423301) # 125
#cap.set(cv2.CAP_PROP_POS_MSEC, 5 + 2 * 60) # 125
#cap.set(cv2.CAP_PROP_POS_MSEC, 182 + 6 * 60 + 417) # 542
#cap.set(cv2.CAP_PROP_POS_MSEC, 182 + 6 * 60 + 417 ) # 959
#cap.set(cv2.CAP_PROP_POS_MSEC, 182 + 6 * 60 + 417 + 415) #1374 4 * 328
#cap.set(cv2.CAP_PROP_POS_MSEC, 182 + 6 * 60 + 417 + 416 + 417) #1791 5 * 328

#Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
ret, frame = cap.read()

#Set grayscale colorspace for the frame. 
gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
crop = gray[63:97, 359:550]
# ret,crop = cv2.threshold(crop,127,255,cv2.THRESH_BINARY)
# ret,crop = cv2.threshold(crop,0,127,cv2.THRESH_BINARY_INV)
crop1 = crop[0:35, 0:24]
crop2 = crop[0:35, 24:24+24]
crop3 = crop[0:35, 71:71+24]
crop4 = crop[0:35, 95:95+24]
crop5 = crop[0:35, 143:143+24]
crop6 = crop[0:35, 167:167+24]
crop7 = gray[63:97, 71:71+24]
crop8 = gray[63:97, 95:95+24]

current_frame = [crop1, crop2, crop3, crop4, crop5, crop6, crop7, crop8]
if prev_frame is not None:
  diff_idx = check_frame_simularity(current_frame, prev_frame)
  if len(diff_idx) > 0:
    #Cut the video extension to have the name of the video
    my_video_name = video_name.split(".")[0]

    #Display the resulting frame
    # cv2.imshow(my_video_name+' frame '+ str(frame_seq),gray)

    #Set waitKey 
    # cv2.waitKey()

    #Store this frame to an image
    print('save')
    cv2.imwrite(os.path.join('/content/frames/', my_video_name+'_frame_'+str(frame_seq)+'.jpg'), crop)
    for i in diff_idx:
      current_img = current_frame[i]
      prev_img = prev_frame[i]
      cv2.imwrite(os.path.join('/content/frames/', my_video_name+'_prv_'+str(frame_seq)+'_' + str(i) + '.jpg'), prev_img)
      cv2.imwrite(os.path.join('/content/frames/', my_video_name+'_cur_'+str(frame_seq)+'_' + str(i) + '.jpg'), current_img)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_1.jpg',crop1)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_2.jpg',crop2)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_3.jpg',crop3)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_4.jpg',crop4)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_5.jpg',crop5)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_6.jpg',crop6)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_7.jpg',crop7)
    # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'_8.jpg',crop8)

prev_frame = current_frame

# When everything done, release the capture
cap.release()
# cv2.destroyAllWindows()

60.0
747578.0
207 12459.633333333333
2.541540815807849e-07
save


## Recognize text

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [ ]:
model_version = "microsoft/trocr-base-printed"
processor = TrOCRProcessor.from_pretrained(model_version)
model = VisionEncoderDecoderModel.from_pretrained(model_version)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=76

In [ ]:
image = cv2.imread('/content/frames/test_frame_18.jpg')

In [ ]:
image2 = cv2.imread('/content/frames/test_frame_19.jpg')

In [ ]:
pixel_values = processor([image, image2], return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['21:33:06', '21:33:08']


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
!rm -rf /content/frames

In [ ]:
!mkdir /content/frames

In [ ]:
save_diff = False
prev_frame = None
current_frame = None
cap = cv2.VideoCapture(os.path.join(video_path, video_name))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(fps)
print(frame_count)
seconds = frame_count / fps
minutes = int(seconds / 60)
print(minutes, seconds)
result = {}
crop_images = []
frame_idxs = []
date_idx = 0
#dates = ['2022-05-22 ', '2022-05-23 ', '2022-05-24 '] # test
#dates = ['2022-05-24 ', '2022-05-25 ', '2022-05-26 '] # train1
dates = ['2022-05-26 ', '2022-05-27 ', '2022-05-28 '] # train2
prev_hours = 0

60.0
1202086.0
333 20034.766666666666


In [ ]:
# for frame_idx in range(2000):
while True: 
  ret, frame = cap.read()
  if ret == False:
    break

  #Set grayscale colorspace for the frame. 
  # gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
  crop = frame[63:97, 359:550]
  # ret,crop_th = cv2.threshold(crop,127,255,cv2.THRESH_BINARY)
  crop1 = crop[0:35, 0:24]
  crop2 = crop[0:35, 24:24+24]
  crop3 = crop[0:35, 71:71+24]
  crop4 = crop[0:35, 95:95+24]
  crop5 = crop[0:35, 143:143+24]
  crop6 = crop[0:35, 167:167+24]
  # crop7 = gray[63:97, 71:71+24]
  # crop8 = gray[63:97, 95:95+24]

  # crop = frame[115:174, 476:734]
  # crop1 = crop[0:59, 0:33]
  # crop2 = crop[0:59, 33:67]
  # crop3 = crop[0:59, 98:130]
  # crop4 = crop[0:59, 130:162]
  # crop5 = crop[0:59, 194:226]
  # crop6 = crop[0:59, 226:257]
  current_frame = [crop1, crop2, crop3, crop4, crop5, crop6]
  # cv2.imwrite(os.path.join('/content/frames/', '_frame_'+str(frame_idx)+'.jpg'), crop)
  if prev_frame is not None:
    diff_idxs = check_frame_simularity(current_frame, prev_frame)
    if len(diff_idxs) > 0:
      #Cut the video extension to have the name of the video
      my_video_name = video_name.split(".")[0]
      crop_images.append(crop)
      frame_idxs.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
      #Display the resulting frame
      # cv2.imshow(my_video_name+' frame '+ str(frame_seq),gray)

      #Set waitKey 
      # cv2.waitKey()
      # crop_color = cv2.cvtColor(crop, cv2.COLOR_GRAY2RGB)
      # cap.get(cv2.CV_CAP_PROP_POS_FRAMES)
      if len(crop_images) > 10:
        pixel_values = processor(crop_images, return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
        idx = 0
        for text_idx in generated_text:
          text_times = text_idx.split(':')
          if len(text_times) == 3:
            new_text_times = []
            for text_time in text_times:
              new_text_times.append(text_time.strip())
            
            if len(new_text_times[0]) == 2:
              hours = int(new_text_times[0])
              if hours < prev_hours and date_idx <= 1:
                date_idx += 1
              prev_hours = hours
            
            result_key = dates[date_idx] + ':'.join(new_text_times)
            result[result_key] = frame_idxs[idx]
          idx += 1
          
        print(generated_text)
        crop_images = []
        frame_idxs = []

      #Store this frame to an image
      
      if save_diff:
        cv2.imwrite(os.path.join('/content/frames/', '_frame_'+str(frame_idx)+'.jpg'), crop)
        for diff_idx in diff_idxs:
          current_img = current_frame[diff_idx]
          prev_img = prev_frame[diff_idx]
          cv2.imwrite(os.path.join('/content/frames/', '_prv_'+str(frame_idx)+'_' + str(i) + '.jpg'), prev_img)
          cv2.imwrite(os.path.join('/content/frames/', '_cur_'+str(frame_idx)+'_' + str(i) + '.jpg'), current_img)

  prev_frame = current_frame



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['08:02:48', '08:02:49', '08:02:50', '08:02:51', '08:02:52', '08:02:53', '08:02:54', '08:02:55', '08:02:56', '08:02:57', '08:03:12']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['08:03:13', '08:03:14', '08:03:15', '08:03:16', '08:03:17', '08:03:18', '08:03:19', '08:03:20', '08:03:21', '08:03:22', '08:03:23']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

['07:44:35', '07:44:36', '07:44:37', '07:44:38', '07:44:39', '07:44:40', '07:44:41', '07:44:42', '07:44:43', '07:44:44', '07:44:45']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['07:44:46', '07:44:47', '07:44:48', '07:44:49', '07:44:50', '07:44:51', '07:44:52', '07:44:53', '07:44:54', '07:46:19', '07:46:20']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['07:46:21', '07:46:22', '07:46:23', '07:46:24', '07:46:25', '07:46:26', '07:46:27', '07:46:28', '07:46:29', '07:46:30', '07:46:31']


In [ ]:
# When everything done, release the capture
cap.release()
# cv2.destroyAllWindows()

In [ ]:
len(result)

20429

In [ ]:
len(result)

20429

In [ ]:
'2022-05-28 20:34:41' in result

True

In [ ]:
result['2022-05-28 08:30:12']

254041.0

In [ ]:
new_result = {}

In [ ]:
new_result

{'2022-05-27 08:04:33': 248681.0,
 '2022-05-27 08:04:34': 248691.0,
 '2022-05-27 08:04:35': 248716.0,
 '2022-05-27 08:04:36': 248741.0,
 '2022-05-27 08:04:37': 248766.0,
 '2022-05-27 08:04:38': 248791.0,
 '2022-05-27 08:04:39': 248816.0,
 '2022-05-27 08:04:40': 248841.0,
 '2022-05-27 08:04:41': 248866.0,
 '2022-05-27 08:04:42': 248891.0,
 '2022-05-27 08:04:43': 248916.0,
 '2022-05-27 08:04:44': 248941.0,
 '2022-05-27 08:05:41': 248961.0,
 '2022-05-27 08:05:42': 248972.0,
 '2022-05-27 08:05:43': 248996.0,
 '2022-05-27 08:05:44': 249021.0,
 '2022-05-27 08:05:45': 249046.0,
 '2022-05-27 08:05:46': 249071.0,
 '2022-05-27 08:05:47': 249096.0,
 '2022-05-27 08:05:48': 249121.0,
 '2022-05-27 08:05:49': 249146.0,
 '2022-05-27 08:05:50': 249171.0,
 '2022-05-27 08:05:51': 249196.0,
 '2022-05-27 08:05:52': 249221.0,
 '2022-05-27 08:05:53': 249246.0,
 '2022-05-27 08:05:54': 249271.0,
 '2022-05-27 08:05:55': 249296.0,
 '2022-05-27 08:05:56': 249321.0,
 '2022-05-27 08:12:00': 249341.0,
 '2022-05-27 0

In [ ]:
for key, value in result.items():
  if '2022-05-28' in key:
    key_split1 = key.split(' ')
    time_split = key_split1[1].split(':')
    if (time_split[0] == '08' or time_split[0] == '09' or time_split[0] == '10' or time_split[0] == '11' or 
      time_split[0] == '12' or time_split[0] == '13' or time_split[0] == '14' or time_split[0] == '15' or 
      time_split[0] == '16' or time_split[0] == '17' or time_split[0] == '18' or time_split[0] == '19' or time_split[0] == '20'):
      new_result['2022-05-27 ' + ':'.join(time_split)] = value

In [ ]:
def find_and_set(x):
  return result[x] if x in result else 0

In [ ]:
def find_and_set(x):
  return result[x] if x in result else (new_result[x] if x in new_result else 0)

In [ ]:
df_test = pd.read_csv("/content/test/test.csv")

In [ ]:
df_train = pd.read_csv("/content/train/train.csv")

In [ ]:
df_test['frame'] = df_test['date'].apply(find_and_set)

In [ ]:
df_train['frame'] = df_train['date'].apply(find_and_set)

In [ ]:
df_train.drop(df_train[df_train['id'] < 2525].index, inplace=True)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2250 entries, 1269 to 3518
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        2250 non-null   int64  
 1   date      2250 non-null   object 
 2   employee  2250 non-null   int64  
 3   action    2250 non-null   float64
 4   frame     2250 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 105.5+ KB


In [ ]:
df_test['seconds'] = df_test['frame'] / 25

In [ ]:
df_train['seconds'] = df_train['frame'] / 25

In [ ]:
df_test[df_test['frame'] == 0]

,id,date,frame,seconds
108,243,2022-05-22 15:00:00,0.0,0.0
109,244,2022-05-22 15:00:11,0.0,0.0
149,284,2022-05-22 16:13:27,0.0,0.0
637,772,2022-05-23 12:59:27,0.0,0.0
939,1074,2022-05-23 18:02:00,0.0,0.0
958,1093,2022-05-23 18:44:33,0.0,0.0
961,1096,2022-05-23 18:45:50,0.0,0.0
962,1097,2022-05-23 18:45:54,0.0,0.0
1120,1255,2022-05-24 07:57:41,0.0,0.0


In [ ]:
df_train.to_csv('fix_train2.csv')

In [ ]:
df_test.to_csv('fix_test.csv')

In [ ]:
df_train.to_csv('fix_train.csv')

In [ ]:
df_train = pd.read_csv("/content/fix_train.csv")

In [ ]:
df_train

,id,date,employee,action,frame,seconds
1269,2525,2022-05-26 08:07:11,6,5.0,2141.0,85.64
1270,2526,2022-05-26 08:15:16,1,15.0,2247.0,89.88
1271,2527,2022-05-26 08:15:18,1,4.0,2297.0,91.88
1272,2528,2022-05-26 08:17:36,1,14.0,2807.0,112.28
1273,2529,2022-05-26 08:17:37,1,5.0,2832.0,113.28
...,...,...,...,...,...,...
3514,4770,2022-05-28 07:42:51,0,4.0,499057.0,19962.28
3515,4771,2022-05-28 07:44:27,0,14.0,499217.0,19968.68
3516,4772,2022-05-28 07:44:36,0,3.0,499442.0,19977.68
3517,4773,2022-05-28 07:46:21,0,13.0,499948.0,19997.92


## Get 6 seconds

In [ ]:
df_test = pd.read_csv('/content/test/fix_test.csv')

In [ ]:
start_frame = df_test[df_test['id'] ==775]['frame'].values[0]

In [ ]:
start_frame -= 4 * 25

In [ ]:
start_frame

181091.0

In [ ]:
end_frame = start_frame + 8 * 25

In [ ]:
current_frame = start_frame


In [ ]:
cap = cv2.VideoCapture(os.path.join(video_path, video_name))
cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

True

In [ ]:
for frame_idx in range(6):
  ret, frame = cap.read()
  if ret == False:
    break

  cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        
  cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  current_frame += 25



In [ ]:
cap.release()

## Try Detect

In [6]:
def draw_bbox(image, bboxes, tracking=False):   
    image_h, image_w, _ = image.shape

    for i, bbox in enumerate(bboxes):
        coor = np.array(bbox[:4], dtype=np.int32)
        score = bbox[4]
        class_ind = int(bbox[5])
        bbox_color = (255,0,0)
        bbox_thick = int(0.6 * (image_h + image_w) / 1000)
        if bbox_thick < 1: bbox_thick = 1
        fontScale = 0.75 * bbox_thick
        (x1, y1), (x2, y2) = (coor[0], coor[1]), (coor[2], coor[3])

        # 1167.5      275.52      1213.3      402.88
        # put object rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), bbox_color, bbox_thick*2)
    return image

In [14]:
def to_xy_min(x, y, width, height):
  return [x, y, x+width, y + height]

In [15]:
def to_wh(x_min, y_min, x_max, y_max):
  return [x_min, y_min, x_max - x_min, y_max - y_min]

In [16]:
def convert_seconds(seconds):
  mins_div = int(seconds / 60)
  hours_div = int(seconds / 60 / 60)
  # mins_diff = 
  # print(mins_diff)
  seconds_diff = seconds - mins_div * 60
  mins = mins_div - hours_div * 60 if hours_div > 0 else mins_div
  return str(hours_div) + ':' + str(mins) + ':' + str(seconds_diff)

In [7]:
import torch

model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5m6')
model_yolov5.classes = [0]


/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: ipython not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab

  0%|          | 0.00/69.0M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5m6 summary: 378 layers, 35704908 parameters, 0 gradients
Adding AutoShape... 


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [9]:
model_yolov5.to(device)

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 48, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
  

In [10]:
# Definition of the parameters
max_cosine_distance = 0.7
nn_budget = None

In [17]:
#initialize deep sort object
model_filename = '/content/TensorFlow-2.x-YOLOv3/model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)

In [18]:
def detect_direction(traking_cords):
  '''
  return 0 if direction from right to left otherwise return 1 from left to rigth
  '''
  # count_up = 0
  # count_down = 0
  # prev_cords = None
  # for cords in traking_cords:
  #   if prev_cords is not None:
  #     if prev_cords[0] < cords[0]:
  #       count_down += 1
  #     else:
  #       count_up += 1
  #   prev_cords = cords
  
  # return 0 if count_down > count_up else 1
  if len(traking_cords) >= 2:
    #print('First second: ', traking_cords[0][0], traking_cords[-1][0])
    first = traking_cords[0][0]
    last = traking_cords[-1][0]
    return 0 if first > last else 1
  return -1

In [19]:
# door_cords = [
#     [116,  528,  600,  1075],
#     [770,  304,  960,  730],
#     [1141, 240, 1213, 415],
#     [1205, 225, 1253, 343],
#     [1302, 234, 1324, 345],
#     [1277, 308, 1321, 409],
#     [1242, 354, 1326, 640],
#     [1200, 400, 1326, 350],
#     [1100, 431, 1322, 817]]
door_cords = [
    [116,  528,  600,  1075],
    [770,  304,  1050,  750],
    [1141, 240, 1213, 415],
    [1205, 225, 1270, 346],
    [1302, 234, 1324, 345],
    [1277, 308, 1321, 409],
    [1242, 354, 1326, 640],
    [1200, 400, 1326, 350],
    [1100, 431, 1322, 817]]
def detect_door(cords):
  x1, y1, x2, y2 = cords
  center_x = int(x1 + (x2 - x1) / 2)
  center_y = int(y1 + (y2 - y1) / 2)
  print(center_x, center_y)
  door_idx = 0
  for door_cord in door_cords:
    door_x1, door_y1, door_x2, door_y2 = door_cord
    if center_x > door_x1 and center_y > door_y1 and center_x < door_x2 and center_y < door_y2:
      return door_idx + 1
    door_idx += 1
  return -1

def filter_cors(door_cord, cords):
  door_x1, door_y1, door_x2, door_y2 = door_cord
  filtered_cords = []
  for cord in cords:
    x1, y1, x2, y2 = cord
    center_x = int(x1 + (x2 - x1) / 2)
    center_y = int(y1 + (y2 - y1) / 2)
    if center_x > door_x1 and center_y > door_y1 and center_x < door_x2 and center_y < door_y2:
      filtered_cords.append(cord)
  return filtered_cords

In [20]:
def find_door(traking_cords):
  for traking_cord in traking_cords:
    #print(traking_cord)
    door = detect_door(traking_cord)
    #print(door)
    if door != -1:
      return door
  return 0

In [21]:
def find_cords_by_door(traking_cords):
  cords_by_doors = []
  max_cords = 0
  max_idx = -1
  idx = 0
  for door_cord in door_cords:
    filtered_cords = filter_cors(door_cord, traking_cords)
    cords_by_doors.append(filtered_cords)
    if (len(filtered_cords) > max_cords):
      #print('door: ', idx, len(filtered_cords))
      max_cords = len(filtered_cords)
      max_idx = idx
    idx += 1
  return (cords_by_doors[max_idx], max_idx + 1)

In [22]:
def find_door_by_objects(traking_objects):
  
  for key, traking_object_cords in traking_objects.items():
    #print(key)
    #print(traking_object_cords)
    if len(traking_objects) > 1 and key == -1:
      continue
    #door_1 = find_door(traking_object_cords)
    filtered_cords, door = find_cords_by_door(traking_object_cords)
    direction_1 = detect_direction(filtered_cords)
    # traking_object_cords.reverse()
    #door_2 = find_door(filtered_cords)
    #direction_2 = detect_direction(traking_object_cords[:5])
    if door != 0:
      return (door, direction_1)
  return (0, 0)

In [23]:
def detect_tracking_object(frames, frame_id, id):
  tracker = Tracker(metric)
  traking_objects = {}
  traking_objects[-1] = []
  original_boxes = []
  last_frame = None
  for frame in frames:
    original_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model_yolov5(np.array(frame))
    if results.xyxy[0].shape != torch.Size([0, 6]):
      bboxes = results.xyxy[0].cpu().numpy()
      #print(bboxes)
      boxes, scores, names = [], [], []
      for bbox in bboxes:
        boxes.append([bbox[0].astype(int), bbox[1].astype(int), bbox[2].astype(int)-bbox[0].astype(int), bbox[3].astype(int)-bbox[1].astype(int)])
        original_boxes.append(bbox[:4])
        scores.append(bbox[4])
        names.append('person')
      
      if False:
        # Obtain all the detections for the given frame.
        boxes = np.array(boxes) 
        names = np.array(names)
        scores = np.array(scores)
        features = np.array(encoder(original_frame, boxes))
        detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(boxes, scores, names, features)]

        # Pass detections to the deepsort object and obtain the track information.
        tracker.predict()
        tracker.update(detections)

        # Obtain info from the tracks
        tracked_bboxes = []
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 5:
                continue 
            bbox = track.to_tlbr() # Get the corrected/predicted bounding box
            class_name = track.get_class() #Get the class name of particular object
            tracking_id = track.track_id # Get the ID for the particular track
            index = 0 #key_list[val_list.index(class_name)] # Get predicted object index by object name
            #print(tracking_id, bbox)
            tracked_bboxes.append(bbox.tolist() + [tracking_id, index]) # Structure data, that we could use it with our draw_bbox function
            if tracking_id in traking_objects:
              traking_objects[tracking_id].append(bbox.tolist())
            else:
              traking_objects[tracking_id] = [bbox.tolist()]
      
      # if (len(tracked_bboxes) <= 1):
      #   for original_box in original_boxes:
      #     traking_objects[-1].append(original_box)
      # draw detection on frame
      #image = draw_bbox(original_frame, tracked_bboxes, tracking=True)
    last_frame = frame

  if (len(traking_objects) == 1):
    traking_objects[1] = original_boxes

  if False and last_frame is not None:
    cv2.imwrite(os.path.join('/content/frames/', 'framef_'+str(frame_id)+'.jpg'), frames[0])
    cv2.imwrite(os.path.join('/content/frames/', 'framel_'+str(frame_id)+'.jpg'), last_frame)
  
  return find_door_by_objects(traking_objects)

In [25]:
df_test = pd.read_csv('/content/test/fix_test.csv')

In [26]:
all_frames = df_test['frame'].values
all_ids= df_test['id'].values

In [27]:
video_path = '/content/test'
video_name = 'test.avi'

In [28]:
current_idx = 0
current_frame_id = all_frames[current_idx]
current_id = all_ids[current_idx]
result = []
frames = []

In [29]:
cap = cv2.VideoCapture(os.path.join(video_path, video_name))
while True:
# for i in range(10000):
  ret, frame = cap.read()
  frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
  #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
  if ret == False:
    break

  if frame_id % 1000 == 0:
    print(frame_id)

  if (len(frames) > 200):
    frames = frames[-150:]
  frames.append(frame)
    
  need_detect = frame_id > current_frame_id + 25 * 3
  if need_detect:
    # print(len(frames))
    last_frames = frames[-150:]
    res1 = detect_tracking_object(last_frames[50:100], current_frame_id, current_id)
    if res1[0] == 0 or res1[1] == -1:
      res2 = detect_tracking_object(frames[25:125], current_frame_id, current_id)
      if res2[0] == 0 or res2[1] == -1:
        res3 = detect_tracking_object(frames[:], current_frame_id, current_id)
        result.append(res3)
      else:
        result.append(res2)
    else:
      result.append(res1)
    # result.append(detect_tracking_object(frames[-150:], current_frame_id, current_id))
    
    current_idx += 1
    current_frame_id = all_frames[current_idx]
    current_id = all_ids[current_idx]
    
    while current_frame_id == 0:
      result.append((0, 0))
      current_idx += 1
      current_frame_id = all_frames[current_idx]
      current_id = all_ids[current_idx]
  # if int(current_frame_id) == int(frame_id) or int(current_frame_id) - 25 == int(frame_id) or int(current_frame_id)-50 == int(frame_id) or int(current_frame_id)+25 == int(frame_id):
  #   crop_object(frame, frame_id)

  # cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  # cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
  # current_frame += 1
  #cap.set(cv2.CAP_PROP_POS_MSEC, seconds)
  #seconds += 1
  
cap.release()

1000.0
2000.0
3000.0
4000.0
5000.0
6000.0
7000.0
8000.0
9000.0
10000.0
11000.0
12000.0
13000.0
14000.0
15000.0
16000.0
17000.0
18000.0
19000.0
20000.0
21000.0
22000.0
23000.0
24000.0
25000.0
26000.0
27000.0
28000.0
29000.0
30000.0
31000.0
32000.0
33000.0
34000.0
35000.0
36000.0
37000.0
38000.0
39000.0
40000.0
41000.0
42000.0
43000.0
44000.0
45000.0
46000.0
47000.0
48000.0
49000.0
50000.0
51000.0
52000.0
53000.0
54000.0
55000.0
56000.0
57000.0
58000.0
59000.0
60000.0
61000.0
62000.0
63000.0
64000.0
65000.0
66000.0
67000.0
68000.0
69000.0
70000.0
71000.0
72000.0
73000.0
74000.0
75000.0
76000.0
77000.0
78000.0
79000.0
80000.0
81000.0
82000.0
83000.0
84000.0
85000.0
86000.0
87000.0
88000.0
89000.0
90000.0
91000.0
92000.0
93000.0
94000.0
95000.0
96000.0
97000.0
98000.0
99000.0
100000.0
101000.0
102000.0
103000.0
104000.0
105000.0
106000.0
107000.0
108000.0
109000.0
110000.0
111000.0
112000.0
113000.0
114000.0
115000.0
116000.0
117000.0
118000.0
119000.0
120000.0
121000.0
122000.0
123000.0
1

IndexError: ignored

In [30]:
result

[(5, 0),
 (4, 0),
 (3, 0),
 (4, 1),
 (0, 0),
 (4, 1),
 (9, 1),
 (9, 0),
 (9, 1),
 (9, 0),
 (4, 0),
 (4, 0),
 (4, 0),
 (0, 0),
 (0, 0),
 (5, 0),
 (4, 0),
 (4, 0),
 (4, 0),
 (4, 1),
 (4, 0),
 (3, 0),
 (3, 0),
 (2, 1),
 (2, 0),
 (2, 1),
 (4, 0),
 (4, 0),
 (4, 0),
 (4, 0),
 (4, 0),
 (4, 1),
 (2, 0),
 (5, 0),
 (4, 0),
 (4, 0),
 (0, 0),
 (4, 0),
 (4, 0),
 (4, 1),
 (4, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (4, 1),
 (4, 1),
 (4, 1),
 (4, 1),
 (4, 1),
 (4, 1),
 (4, 0),
 (4, 1),
 (4, 0),
 (4, 1),
 (4, 0),
 (4, 0),
 (4, 1),
 (4, 1),
 (9, 0),
 (2, 0),
 (2, 1),
 (2, 1),
 (9, 1),
 (4, 1),
 (5, 0),
 (5, 1),
 (5, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (4, 0),
 (4, 0),
 (4, 0),
 (4, 1),
 (3, 0),
 (2, 0),
 (0, 0),
 (2, 0),
 (2, 1),
 (9, 1),
 (9, 0),
 (4, 0),
 (4, 0),
 (4, 1),
 (4, 1),
 (4, 0),
 (4, 0),
 (5, 1),
 (4, 0),
 (4, 0),
 (6, -1),
 (2, 0),
 (2, 1),
 (9, 1),
 (4, 0),
 (5, 1),
 (5, 1),
 (5, 1),
 (4, 1),
 (2, 0),
 (2, 1),
 (4, 0),
 (4, 1),
 (4, 0),
 (4, -1),
 (4, 1),
 (0, 0),
 (0, 0),
 (4, 0),

In [ ]:
# 0,135,2022-05-22 11:51:06,5804.0,232.16 (5, 0) 
# 1,136,2022-05-22 11:51:25,6279.0,251.16 (4, 0) 
# 2,137,2022-05-22 12:04:27,6949.0,277.96 (3, 0) 
# 3,138,2022-05-22 12:07:56,7214.0,288.56 (3, 1) 
# 4,139,2022-05-22 12:08:10,7564.0,302.56 (4, 0)
# 5,140,2022-05-22 12:08:57,7700.0,308.0  (4, 1) 
# 6,141,2022-05-22 12:09:23,8349.0,333.96 (8, 1)
# 7,142,2022-05-22 12:11:28,8915.0,356.6  (8, 0)
# 8,143,2022-05-22 12:11:40,9214.0,368.56 (8, 1)
# 9,144,2022-05-22 12:11:45,9339.0,373.56 (8, 0)
# 10,145,2022-05-22 12:12:31,10489.0,419.56 (4, 0)

In [ ]:
def store_tracking_object(frames, frame_id):
  tracker = Tracker(metric)
  traking_objects = {}
  # traking_objects[-1] = []
  original_boxes = []
  last_frame = None
  for frame in frames:
    original_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model_yolov5(np.array(frame))
    if results.xyxy[0].shape != torch.Size([0, 6]):
      bboxes = results.xyxy[0].cpu().numpy()
      #print(bboxes)
      boxes, scores, names = [], [], []
      for bbox in bboxes:
        boxes.append([bbox[0].astype(int), bbox[1].astype(int), bbox[2].astype(int)-bbox[0].astype(int), bbox[3].astype(int)-bbox[1].astype(int)])
        original_boxes.append(bbox[:4])
        scores.append(bbox[4])
        names.append('person')
      
      # Obtain all the detections for the given frame.
      boxes = np.array(boxes) 
      names = np.array(names)
      scores = np.array(scores)
      features = np.array(encoder(original_frame, boxes))
      detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(boxes, scores, names, features)]

      # Pass detections to the deepsort object and obtain the track information.
      tracker.predict()
      tracker.update(detections)

      # Obtain info from the tracks
      tracked_bboxes = []
      for track in tracker.tracks:
          if not track.is_confirmed() or track.time_since_update > 5:
              continue 
          bbox = track.to_tlbr() # Get the corrected/predicted bounding box
          class_name = track.get_class() #Get the class name of particular object
          tracking_id = track.track_id # Get the ID for the particular track
          index = 0 #key_list[val_list.index(class_name)] # Get predicted object index by object name
          #print(tracking_id, bbox)
          tracked_bboxes.append(bbox.tolist() + [tracking_id, index]) # Structure data, that we could use it with our draw_bbox function
          if tracking_id in traking_objects:
            traking_objects[tracking_id].append(bbox.tolist())
          else:
            traking_objects[tracking_id] = [bbox.tolist()]
      
      # if (len(tracked_bboxes) <= 1):
      #   for original_box in original_boxes:
      #     traking_objects[-1].append(original_box)
      # draw detection on frame
      #image = draw_bbox(original_frame, tracked_bboxes, tracking=True)
    last_frame = frame

  if (len(traking_objects) == 1):
    traking_objects[1] = original_boxes
  
  cords = [[0,0]] * 100
  idx = 0
  detect = False
  if 1 in traking_objects:
    detect = len(traking_objects[1]) > 0
    for track_cords in traking_objects[1]:
      cords[idx] = [track_cords[0],track_cords[1]]
      idx += 1
      if idx == 100:
        break
  if detect == False and len(traking_objects) > 0:
    print(tracking_id)
  return (detect, cords)

In [ ]:
!rm -rf frames
!mkdir frames

In [ ]:
df_train = pd.read_csv('/content/train/fix_train2.csv')

In [ ]:
all_frames = df_train['frame'].values

In [ ]:
all_action = df_train['action'].values

In [ ]:
video_path = '/content/train'
video_name = 'train2.avi'

In [ ]:
def ohe(val, l):
  res = [0] * l
  res[int(val) - 1] = 1
  return res

In [ ]:
current_idx = 0
current_frame_id = all_frames[current_idx]
current_action = all_action[current_idx]
result = []
frames = []
train_cords_x = []
train_cords_y = []

In [ ]:
cap = cv2.VideoCapture(os.path.join(video_path, video_name))
while True:
# for i in range(10000):
  ret, frame = cap.read()
  frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
  #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
  if ret == False:
    break

  if frame_id % 1000 == 0:
    print(frame_id)

  if (len(frames) > 200):
    frames = frames[-150:]
  frames.append(frame)
    
  need_detect = frame_id > current_frame_id + 25 * 3
  if need_detect:
    # print(len(frames))
    # result.append(detect_tracking_object(frames[-100:], current_frame_id))
    detected, cords = store_tracking_object(frames[-150:], current_frame_id)
    if detected:
      train_cords_x.append(cords)
      train_cords_y.append(ohe(current_action, 20))
    else:
      print('Not detected ', current_frame_id)
    #frames = frames[-100:]
    current_idx += 1
    current_frame_id = all_frames[current_idx]
    current_action = all_action[current_idx]
    
    while current_frame_id == 0:
      # result.append((0, 0))
      current_idx += 1
      current_frame_id = all_frames[current_idx]
      current_action = all_action[current_idx]
  # if int(current_frame_id) == int(frame_id) or int(current_frame_id) - 25 == int(frame_id) or int(current_frame_id)-50 == int(frame_id) or int(current_frame_id)+25 == int(frame_id):
  #   crop_object(frame, frame_id)

  # cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  # cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
  # current_frame += 1
  #cap.set(cv2.CAP_PROP_POS_MSEC, seconds)
  #seconds += 1
  
cap.release()

1000.0
2000.0
Not detected  2247.0
Not detected  2297.0
Not detected  2807.0
Not detected  2832.0
3000.0
Not detected  3621.0
4000.0
Not detected  4486.0
Not detected  4587.0
Not detected  4731.0
Not detected  4806.0
5000.0
Not detected  5026.0
Not detected  5152.0
Not detected  5331.0
Not detected  5381.0
6000.0
Not detected  6426.0
Not detected  6476.0
Not detected  6711.0
Not detected  6861.0
7000.0
Not detected  7011.0
Not detected  7036.0
8000.0
9000.0
10000.0
11000.0
12000.0
Not detected  11926.0
13000.0
Not detected  13052.0
Not detected  13052.0
Not detected  13321.0
14000.0
15000.0
Not detected  15016.0
Not detected  15041.0
Not detected  15141.0
Not detected  15166.0
Not detected  15402.0
Not detected  15427.0
Not detected  15427.0
16000.0
Not detected  16346.0
17000.0
18000.0
Not detected  18826.0
19000.0
20000.0
21000.0
22000.0
23000.0
Not detected  23781.0
Not detected  23806.0
24000.0
Not detected  24006.0
Not detected  24056.0
25000.0
26000.0
Not detected  26601.0
27000.

IndexError: ignored

In [ ]:
len(train_cords_x)

1705

In [ ]:
len(train_cords_y)

1705

In [137]:
only_first_last_x = []
for cords in train_cords_x:
  first_cord = cords[0]
  last_cord = cords[0]
  for i in range(1, len(cords) - 1):
    if (cords[i][0] == 0 and cords[i][1] == 0):
      break
    last_cord = cords[i]
  only_first_last_x.append([first_cord, last_cord])

In [148]:
only_first_last_y = []
for yy in train_cords_y:
  val = np.argmax(yy)
  only_first_last_y.append(ohe(val + 2, 20))

In [97]:
only_first_last_x = []
for cords in train_cords_x:
  first_cord = cords[0]
  last_cord = cords[0]
  for i in range(1, len(cords) - 1):
    if (cords[i][0] == 0 and cords[i][1] == 0):
      break
    last_cord = cords[i]
  only_first_last_x.append([first_cord[0], first_cord[1], last_cord[0], last_cord[1] ])

In [150]:
train_x = np.array(only_first_last_x)
train_y = np.array(only_first_last_y)

In [100]:
train_x.shape

(1705, 4)

In [151]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(train_x, train_y, train_size=0.80, shuffle=True, random_state=42)

In [64]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import concatenate, Dropout, Dense, LSTM, Conv1D, BatchNormalization, Flatten, MaxPooling1D, Activation, Input, Reshape


In [142]:
df_train[df_train['action']  == 0]

,Unnamed: 0,id,date,employee,action,frame,seconds
105,1374,2630,2022-05-26 10:07:56,0,0.0,27826.0,1113.04
142,1411,2667,2022-05-26 10:40:57,0,0.0,48471.0,1938.84
143,1412,2668,2022-05-26 10:41:12,0,0.0,48726.0,1949.04
150,1419,2675,2022-05-26 10:42:13,0,0.0,49932.0,1997.28
166,1435,2691,2022-05-26 11:05:26,0,0.0,54556.0,2182.24
849,2118,3374,2027-05-27 00:18:52,0,0.0,0.0,0.00
857,2126,3382,2027-05-27 01:58:29,0,0.0,0.0,0.00


In [107]:
model = Sequential()
model.add(Input(shape=4))

model.add(Dense(1000))
model.add(Dense(512))

model.add(Dense(19, activation='softmax'))

model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [152]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(2, 2)))
model.add(Dropout(0.1))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
#model.add(Dropout(0.1))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))

model.add(Flatten())
model.add(Dense(1000))
model.add(Dense(512))

model.add(Dense(20, activation='softmax'))

model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [159]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_validation, Y_validation))

Epoch 1/100
43/43 [==============================] - 0s 6ms/step - loss: 1.2812 - accuracy: 0.5352 - val_loss: 2.7821 - val_accuracy: 0.5073
Epoch 2/100
43/43 [==============================] - 0s 5ms/step - loss: 1.2675 - accuracy: 0.5535 - val_loss: 2.8026 - val_accuracy: 0.4839
Epoch 3/100
43/43 [==============================] - 0s 4ms/step - loss: 1.2949 - accuracy: 0.5425 - val_loss: 2.7912 - val_accuracy: 0.4633
Epoch 4/100
43/43 [==============================] - 0s 4ms/step - loss: 1.2588 - accuracy: 0.5301 - val_loss: 2.7919 - val_accuracy: 0.5279
Epoch 5/100
43/43 [==============================] - 0s 4ms/step - loss: 1.2448 - accuracy: 0.5447 - val_loss: 2.9058 - val_accuracy: 0.4809
Epoch 6/100
43/43 [==============================] - 0s 4ms/step - loss: 1.2833 - accuracy: 0.5477 - val_loss: 2.8039 - val_accuracy: 0.5161
Epoch 7/100
43/43 [==============================] - 0s 4ms/step - loss: 1.3035 - accuracy: 0.5403 - val_loss: 2.7954 - val_accuracy: 0.5191
Epoch 8/100
4

In [160]:
model.save_weights('cords_weight.h5')

In [110]:
model.load_weights('cords_weight.h5')

In [94]:
df_test = pd.read_csv('/content/test/fix_test.csv')

In [112]:
all_frames = df_test['frame'].values
all_ids= df_test['id'].values

In [96]:
video_path = '/content/test'
video_name = 'test.avi'

In [163]:
def detect_tracking_object2(frames, frame_id, id):
  tracker = Tracker(metric)
  traking_objects = {}
  # traking_objects[-1] = []
  original_boxes = []
  last_frame = None
  for frame in frames:
    original_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model_yolov5(np.array(frame))
    if results.xyxy[0].shape != torch.Size([0, 6]):
      bboxes = results.xyxy[0].cpu().numpy()
      #print(bboxes)
      boxes, scores, names = [], [], []
      for bbox in bboxes:
        boxes.append([bbox[0].astype(int), bbox[1].astype(int), bbox[2].astype(int)-bbox[0].astype(int), bbox[3].astype(int)-bbox[1].astype(int)])
        original_boxes.append(bbox[:4])
        scores.append(bbox[4])
        names.append('person')
      
      # Obtain all the detections for the given frame.
      boxes = np.array(boxes) 
      names = np.array(names)
      scores = np.array(scores)
      features = np.array(encoder(original_frame, boxes))
      detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(boxes, scores, names, features)]

      # Pass detections to the deepsort object and obtain the track information.
      tracker.predict()
      tracker.update(detections)

      # Obtain info from the tracks
      tracked_bboxes = []
      for track in tracker.tracks:
          if not track.is_confirmed() or track.time_since_update > 5:
              continue 
          bbox = track.to_tlbr() # Get the corrected/predicted bounding box
          class_name = track.get_class() #Get the class name of particular object
          tracking_id = track.track_id # Get the ID for the particular track
          index = 0 #key_list[val_list.index(class_name)] # Get predicted object index by object name
          #print(tracking_id, bbox)
          tracked_bboxes.append(bbox.tolist() + [tracking_id, index]) # Structure data, that we could use it with our draw_bbox function
          if tracking_id in traking_objects:
            traking_objects[tracking_id].append(bbox.tolist())
          else:
            traking_objects[tracking_id] = [bbox.tolist()]
      
      # if (len(tracked_bboxes) <= 1):
      #   for original_box in original_boxes:
      #     traking_objects[-1].append(original_box)
      # draw detection on frame
      #image = draw_bbox(original_frame, tracked_bboxes, tracking=True)
    last_frame = frame

  if (len(traking_objects) == 1):
    traking_objects[1] = original_boxes
  
  if 1 in traking_objects:
    cords = traking_objects[1]
    
    if len(cords) > 0:
      first = cords[0]
      last = cords[-1]
      cord_for_predict = np.array([[first[0], first[1]], [last[0], last[1]]])
      cord_for_predict = cord_for_predict.reshape(-1, 2, 2)
      predict = model.predict(cord_for_predict)
      return np.argmax(predict) + 1
  print('Not detect ', id)
  return 4

In [161]:
current_idx = 0
current_frame_id = all_frames[current_idx]
current_id = all_ids[current_idx]
result = []
frames = []

In [162]:
cap = cv2.VideoCapture(os.path.join(video_path, video_name))
# while True:
for i in range(10000):
  ret, frame = cap.read()
  frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
  #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
  if ret == False:
    break

  if frame_id % 1000 == 0:
    print(frame_id)

  if (len(frames) > 200):
    frames = frames[-150:]
  frames.append(frame)
    
  need_detect = frame_id > current_frame_id + 25 * 3
  if need_detect:
    # print(len(frames))
    result.append(detect_tracking_object2(frames[-150:], current_frame_id, current_id))
    
    current_idx += 1
    current_frame_id = all_frames[current_idx]
    current_id = all_ids[current_idx]
    
    while current_frame_id == 0:
      result.append(4)
      current_idx += 1
      current_frame_id = all_frames[current_idx]
      current_id = all_ids[current_idx]
  # if int(current_frame_id) == int(frame_id) or int(current_frame_id) - 25 == int(frame_id) or int(current_frame_id)-50 == int(frame_id) or int(current_frame_id)+25 == int(frame_id):
  #   crop_object(frame, frame_id)

  # cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  # cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
  # current_frame += 1
  #cap.set(cv2.CAP_PROP_POS_MSEC, seconds)
  #seconds += 1
  
cap.release()

1000.0
2000.0
3000.0
4000.0
5000.0
[array([     1296.4,      265.17,      1324.2,      345.02], dtype=float32), array([     1297.3,       266.5,      1323.5,      344.09], dtype=float32), array([     1296.5,      265.86,      1321.7,      345.42], dtype=float32), array([     1295.2,      265.77,      1321.3,      345.52], dtype=float32), array([     1289.8,      265.43,      1317.5,      344.67], dtype=float32), array([     1285.8,       271.5,      1315.9,      346.05], dtype=float32), array([       1284,      270.76,      1316.6,      344.99], dtype=float32), array([       1281,      265.29,      1305.5,      333.99], dtype=float32), array([       1279,      265.22,      1303.2,      334.39], dtype=float32), array([     1276.5,      266.14,      1301.9,      337.11], dtype=float32), array([     1274.5,      265.01,      1299.4,      337.18], dtype=float32), array([     1272.3,      265.26,      1300.9,      338.53], dtype=float32), array([     1272.7,      264.63,      1298.9,      3

In [164]:
result

[16, 16, 4, 14, 4, 15, 19, 19, 13, 3]

In [ ]:
# 0,135,2022-05-22 11:51:06,5804.0,232.16 (5, 0) 
# 1,136,2022-05-22 11:51:25,6279.0,251.16 (4, 0) 
# 2,137,2022-05-22 12:04:27,6949.0,277.96 (3, 0) 
# 3,138,2022-05-22 12:07:56,7214.0,288.56 (3, 1) 
# 4,139,2022-05-22 12:08:10,7564.0,302.56 (4, 0)
# 5,140,2022-05-22 12:08:57,7700.0,308.0  (4, 1) 
# 6,141,2022-05-22 12:09:23,8349.0,333.96 (8, 1)
# 7,142,2022-05-22 12:11:28,8915.0,356.6  (8, 0)
# 8,143,2022-05-22 12:11:40,9214.0,368.56 (8, 1)
# 9,144,2022-05-22 12:11:45,9339.0,373.56 (8, 0)
# 10,145,2022-05-22 12:12:31,10489.0,419.56 (4, 0)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121 entries, 0 to 1120
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1121 non-null   int64  
 1   id          1121 non-null   int64  
 2   date        1121 non-null   object 
 3   frame       1121 non-null   float64
 4   seconds     1121 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 43.9+ KB


In [31]:
len(result)

1121

In [32]:
actions = []
def convert_action(result):
  action = 4
  for res in result:
    door, direction = res
    if door == 1:
      action = 1 if direction == 0 or direction == -1 else 11
    elif door == 2:
      action = 2 if direction == 0 or direction == -1 else 12
    elif door == 3:
      action = 3 if direction == 0 or direction == -1 else 13
    elif door == 4:
      action = 4 if direction == 0 or direction == -1 else 14
    elif door == 5:
      action = 5 if direction == 1 or direction == -1 else 15
    elif door == 6:
      action = 6 if direction == 1 or direction == -1 else 16
    elif door == 7:
      action = 7 if direction == 1 or direction == -1 else 17
    elif door == 8:
      action = 8 if direction == 1 or direction == -1 else 18
    elif door == 9:
      action = 9 if direction == 1 or direction == -1 else 19
    actions.append(action)
convert_action(result)

In [33]:
df_test['action'] = actions

In [34]:
df_test

,Unnamed: 0,id,date,frame,seconds,action
0,0,135,2022-05-22 11:51:06,5804.0,232.16,15
1,1,136,2022-05-22 11:51:25,6279.0,251.16,4
2,2,137,2022-05-22 12:04:27,6949.0,277.96,3
3,3,138,2022-05-22 12:07:56,7214.0,288.56,14
4,4,139,2022-05-22 12:08:10,7564.0,302.56,14
...,...,...,...,...,...,...
1116,1116,1251,2022-05-24 07:57:00,310875.0,12435.00,4
1117,1117,1252,2022-05-24 07:57:32,310935.0,12437.40,4
1118,1118,1253,2022-05-24 07:57:34,310985.0,12439.40,4
1119,1119,1254,2022-05-24 07:57:39,311110.0,12444.40,4


In [ ]:
df_test['employee'] = [0 for x in range(len(actions))]

In [ ]:
df_test

,Unnamed: 0,id,date,frame,seconds,action,employee
0,0,135,2022-05-22 11:51:06,5804.0,232.16,15,0
1,1,136,2022-05-22 11:51:25,6279.0,251.16,4,0
2,2,137,2022-05-22 12:04:27,6949.0,277.96,3,0
3,3,138,2022-05-22 12:07:56,7214.0,288.56,13,0
4,4,139,2022-05-22 12:08:10,7564.0,302.56,13,0
...,...,...,...,...,...,...,...
1116,1116,1251,2022-05-24 07:57:00,310875.0,12435.00,14,0
1117,1117,1252,2022-05-24 07:57:32,310935.0,12437.40,14,0
1118,1118,1253,2022-05-24 07:57:34,310985.0,12439.40,14,0
1119,1119,1254,2022-05-24 07:57:39,311110.0,12444.40,14,0


In [50]:
df_solution = pd.read_csv('solution.csv')

In [51]:
df_solution['action'] = actions

In [53]:
employee = df_solution['employee'] 

In [55]:
employee = [2 if x == 0 else x for x in employee]

In [57]:
df_solution['employee'] = employee

In [ ]:
df_solution = df_test[['id', 'employee', 'action']].copy()

In [ ]:
df_solution

,id,employee,action
0,135,0,15
1,136,0,4
2,137,0,3
3,138,0,13
4,139,0,13
...,...,...,...
1116,1251,0,14
1117,1252,0,14
1118,1253,0,14
1119,1254,0,14


In [58]:
df_solution.to_csv('simple_solution.csv', index=False)

## Crop images

In [ ]:
door_cords = [
    [153,  937,  873,  1890],
    [1043, 569,  1363,  1260],
    [1519, 431, 1623, 730],
    [1627, 409, 1677, 607],
    [1717, 423, 1769, 607],
    [1700, 500, 1763, 723],
    [1671, 631, 1759, 1031],
    [1671, 631, 1759, 1031],
    [1511, 775, 1763, 1481]]

In [ ]:
door_cords = [
    [116,  528,  600,  1075],
    [770,  304,  1050,  750],
    [1141, 240, 1213, 415],
    [1205, 225, 1270, 346],
    [1386, 234, 1324, 345],
    [1277, 308, 1321, 409],
    [1242, 354, 1326, 640],
    [1200, 400, 1326, 350],
    [1100, 431, 1322, 817]]

In [ ]:
!rm -rf /content/train_images

In [ ]:
!mkdir /content/train_images/
!mkdir /content/train_images/0
!mkdir /content/train_images/1
!mkdir /content/train_images/2
!mkdir /content/train_images/3
!mkdir /content/train_images/4
!mkdir /content/train_images/5
!mkdir /content/train_images/6
!mkdir /content/train_images/7
!mkdir /content/train_images/8
!mkdir /content/train_images/9
!mkdir /content/train_images/10
!mkdir /content/train_images/11

In [ ]:
def crop_object(frame, frame_id, object_id):
  original_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  results = model(np.array(frame))
  if results.xyxy[0].shape != torch.Size([0, 6]):
    bboxes = results.xyxy[0].cpu().numpy()
    for bbox in bboxes:
      crop_frame = frame[bbox[1].astype(int):bbox[3].astype(int), bbox[0].astype(int):bbox[2].astype(int)]
      cv2.imwrite(os.path.join('/content/train_images/' + str(object_id) + '/', 'frame_crop_'+str(frame_id)+'.jpg'), crop_frame)
    


In [ ]:
def get_door_by_action(action):
  door = -1
  if action == 1 or action == 11:
    door = 0
  elif action == 2 or action == 12:
    door = 1
  elif action == 3 or action == 13:
    door = 2
  elif action == 4 or action == 14:
    door = 3
  elif action == 5 or action == 15:
    door = 4
  elif action == 6 or action == 16:
    door = 5
  elif action == 7 or action == 17:
    door = 6
  elif action == 8 or action == 18:
    door = 7
  elif action == 9 or action == 19:
    door = 8
  return door

In [ ]:
def detect_people_object(frames, frame_id, object_id, door):
  if door == -1:
    return
  #print(frame_id)
  is_detected = False
  for frame in frames:
    original_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(np.array(frame))
    if results.xyxy[0].shape != torch.Size([0, 6]):
      bboxes = results.xyxy[0].cpu().numpy()
      #print('found ' + str(len(bboxes)))
      for bbox in bboxes:
        door_x1, door_y1, door_x2, door_y2 = door_cords[door]
        x1 = bbox[0].astype(int)
        y1 = bbox[1].astype(int)
        x2 = bbox[2].astype(int)
        y2 = bbox[3].astype(int)
        center_x = int(x1 + (x2 - x1) / 2)
        center_y = int(y1 + (y2 - y1) / 2)
        if center_x > door_x1 and center_y > door_y1 and center_x < door_x2 and center_y < door_y2:
          is_detected = True
          #print('detect')
          crop_frame = frame[y1:y2, x1:x2]
          cv2.imwrite(os.path.join('/content/train_images/' + str(object_id) + '/', 'frame_crop_'+str(frame_id)+'.jpg'), crop_frame)
          return
      #if is_detected == False:
      #  cv2.imwrite(os.path.join('/content/', 'frame_crop_'+str(frame_id)+'.jpg'), frame)
  

In [ ]:
df_train1 = pd.read_csv('/content/fix_train2.csv')

In [ ]:
all_frames = df_train1['frame']
all_employee = df_train1['employee']
all_actions = df_train1['action']

In [ ]:
current_idx = 0
current_frame_id = all_frames[current_idx]
current_employee_id = all_employee[current_idx]
current_action_id = all_actions[current_idx]
frames = []

In [ ]:
cap = cv2.VideoCapture(os.path.join('/content/train/', 'train2.avi'))
while True:
# for i in range(10000):
  ret, frame = cap.read()
  frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
  #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
  if ret == False:
    break

  #cv2.imwrite(os.path.join('/content', 'frame_crop_'+str(frame_id)+'.jpg'), frame)
  if frame_id % 1000 == 0:
    print(frame_id)

  if (len(frames) > 200):
    frames = frames[-100:]
  frames.append(frame)
    
  need_detect = frame_id > current_frame_id + 25
  if need_detect:
    # print(len(frames))
    door = get_door_by_action(current_action_id)
    detect_people_object(frames[-100:], current_frame_id, current_employee_id, door)

    current_idx += 1
    current_frame_id = all_frames[current_idx]
    current_employee_id = all_employee[current_idx]
    current_action_id = all_actions[current_idx]
    while current_frame_id == 0:
      current_idx += 1
      current_frame_id = all_frames[current_idx]
      current_employee_id = all_employee[current_idx]
      current_action_id = all_actions[current_idx]
  # if int(current_frame_id) == int(frame_id) or int(current_frame_id) - 25 == int(frame_id) or int(current_frame_id)-50 == int(frame_id) or int(current_frame_id)+25 == int(frame_id):
  #   

  # cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  # cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
  # current_frame += 1
  #cap.set(cv2.CAP_PROP_POS_MSEC, seconds)
  #seconds += 1
  
cap.release()

1000.0
2000.0
3000.0
4000.0
5000.0
6000.0
7000.0
8000.0
9000.0
10000.0
11000.0
12000.0
13000.0
14000.0
15000.0
16000.0
17000.0
18000.0
19000.0
20000.0
21000.0
22000.0
23000.0
24000.0
25000.0
26000.0
27000.0
28000.0
29000.0
30000.0
31000.0
32000.0
33000.0
34000.0
35000.0
36000.0
37000.0
38000.0
39000.0
40000.0
41000.0
42000.0
43000.0
44000.0
45000.0
46000.0
47000.0
48000.0
49000.0
50000.0
51000.0
52000.0
53000.0
54000.0
55000.0
56000.0
57000.0
58000.0
59000.0
60000.0
61000.0
62000.0
63000.0
64000.0
65000.0
66000.0
67000.0
68000.0
69000.0
70000.0
71000.0
72000.0
73000.0
74000.0
75000.0
76000.0
77000.0
78000.0
79000.0
80000.0
81000.0
82000.0
83000.0
84000.0
85000.0
86000.0
87000.0
88000.0
89000.0
90000.0
91000.0
92000.0
93000.0
94000.0
95000.0
96000.0
97000.0
98000.0
99000.0
100000.0
101000.0
102000.0
103000.0
104000.0
105000.0
106000.0
107000.0
108000.0
109000.0
110000.0
111000.0
112000.0
113000.0
114000.0
115000.0
116000.0
117000.0
118000.0
119000.0
120000.0
121000.0
122000.0
123000.0
1

KeyError: ignored

Вырежим один кадр и посмотрим на него

Загрузим исходники нейросети. В примере рассматривается MobileNet

In [ ]:
!unzip train.zip

Archive:  train.zip
   creating: content/train_images/
   creating: content/train_images/9/
   creating: content/train_images/11/
  inflating: content/train_images/11/frame_crop_253429.0.jpg  
  inflating: content/train_images/11/frame_crop_396720.0.jpg  
  inflating: content/train_images/11/frame_crop_358591.0.jpg  
  inflating: content/train_images/11/frame_crop_333405.0.jpg  
  inflating: content/train_images/11/frame_crop_370791.0.jpg  
  inflating: content/train_images/11/frame_crop_289435.0.jpg  
  inflating: content/train_images/11/frame_crop_360636.0.jpg  
  inflating: content/train_images/11/frame_crop_366700.0.jpg  
  inflating: content/train_images/11/frame_crop_352775.0.jpg  
  inflating: content/train_images/11/frame_crop_370865.0.jpg  
  inflating: content/train_images/11/frame_crop_262460.0.jpg  
  inflating: content/train_images/11/frame_crop_336111.0.jpg  
  inflating: content/train_images/11/frame_crop_341700.0.jpg  
  inflating: content/train_images/11/frame_crop_381

Train

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator # работа с изображениями
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam # оптимизаторы
from sklearn.model_selection import train_test_split

In [ ]:
#Генератор изображений
datagen = ImageDataGenerator(
    rescale=1. / 255, #Значения цвета меняем на дробные показания
    width_shift_range=0.1, #Двигаем изображения по ширине при генерации выборки
    height_shift_range=0.1, #Двигаем изображения по высоте при генерации выборки
    
    zoom_range=0.1, #Зумируем изображения при генерации выборки
    fill_mode='nearest', #Заполнение пикселей вне границ ввода
    validation_split=0.2 #Указываем разделение изображений на обучающую и тестовую выборку
)

Found 4225 validated image filenames belonging to 144 classes.


In [ ]:
train_generator = datagen.flow_from_directory('/content/content/train_images', target_size=(224, 224), class_mode="categorical", shuffle=True, subset="training")
validation_generator = datagen.flow_from_directory('/content/content/train_images', target_size=(224, 224), class_mode="categorical", shuffle=True, subset="validation")

Found 1506 images belonging to 12 classes.
Found 371 images belonging to 12 classes.


In [ ]:
source_model = EfficientNetB0(weights=None, include_top=False,
                 input_shape=(224, 224, 3))

In [ ]:
input_shape = (224,224,3)
input = Input(shape=(224, 224, 3))
x = source_model(input)
# x = Flatten()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(12, activation='softmax')(x)
model = Model(inputs=input, outputs=x)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit_generator(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_steps = validation_generator.samples // 32,
    epochs=10,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
47/47 [==============================] - 36s 529ms/step - loss: 2.2785 - accuracy: 0.2117 - val_loss: 2.2580 - val_accuracy: 0.2102
Epoch 2/10
47/47 [==============================] - 23s 489ms/step - loss: 2.0454 - accuracy: 0.2666 - val_loss: 2.2347 - val_accuracy: 0.2159
Epoch 3/10
47/47 [==============================] - 24s 503ms/step - loss: 1.9771 - accuracy: 0.3005 - val_loss: 2.2532 - val_accuracy: 0.1420
Epoch 4/10
47/47 [==============================] - 23s 487ms/step - loss: 1.9598 - accuracy: 0.2999 - val_loss: 2.2586 - val_accuracy: 0.2131
Epoch 5/10
47/47 [==============================] - 23s 486ms/step - loss: 1.8362 - accuracy: 0.3331 - val_loss: 2.2634 - val_accuracy: 0.2102
Epoch 6/10
47/47 [==============================] - 23s 486ms/step - loss: 1.7768 - accuracy: 0.3677 - val_loss: 2.3712 - val_accuracy: 0.0682
Epoch 7/10
47/47 [==============================] - 23s 489ms/step - loss: 1.7508 - accuracy: 0.3826 - val_loss: 4.9552 - val_accuracy: 0.0909

In [ ]:
model.load_weights('EfficientNetB0.h5')

In [ ]:
model.save_weights('EfficientNetB0.h5')

In [ ]:
!mkdir test_images

In [ ]:
def detect_people_object_test(frames, frame_id, id):
  for frame in frames:
    original_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model_yolov5(np.array(frame))
    if results.xyxy[0].shape != torch.Size([0, 6]):
      bboxes = results.xyxy[0].cpu().numpy()

      for bbox in bboxes:        
        x1 = bbox[0].astype(int)
        y1 = bbox[1].astype(int)
        x2 = bbox[2].astype(int)
        y2 = bbox[3].astype(int)
        
        crop_frame = frame[y1:y2, x1:x2]
        cv2.imwrite(os.path.join('/content/test_images', str(id)+'.jpg'), crop_frame)
        resized = cv2.resize(crop_frame, (224,224), interpolation = cv2.INTER_AREA)
        img = resized.reshape(1,224,224,3) # return the image with shaping that TF wants.
        img = preprocess_input(img)
        print(img.shape)
        img = img / 255
        preds = model.predict(img)
        pred = np.argmax(preds)
        print(pred)
        if pred > 0:
          return pred
  return 0
  

In [ ]:
def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
	"""
	@brief      Overlays a transparant PNG onto another image using CV2
	
	@param      background_img    The background image
	@param      img_to_overlay_t  The transparent image to overlay (has alpha channel)
	@param      x                 x location to place the top-left corner of our overlay
	@param      y                 y location to place the top-left corner of our overlay
	@param      overlay_size      The size to scale our overlay to (tuple), no scaling if None
	
	@return     Background image with overlay on top
	"""
	
	bg_img = background_img.copy()
	
	if overlay_size is not None:
		img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)

	# Extract the alpha mask of the RGBA image, convert to RGB 
	b,g,r,a = cv2.split(img_to_overlay_t)
	overlay_color = cv2.merge((b,g,r))
	
	# Apply some simple filtering to remove edge noise
	mask = cv2.medianBlur(a,5)

	h, w, _ = overlay_color.shape
	roi = bg_img[y:y+h, x:x+w]

	# Black-out the area behind the logo in our original ROI
	img1_bg = cv2.bitwise_and(roi.copy(),roi.copy(),mask = cv2.bitwise_not(mask))
	
	# Mask out the logo from the logo image.
	img2_fg = cv2.bitwise_and(overlay_color,overlay_color,mask = mask)

	# Update the original image with our new ROI
	bg_img[y:y+h, x:x+w] = cv2.add(img1_bg, img2_fg)

	return bg_img

In [ ]:
def merge_images(frames, frame_id, action):
  if len(files) > 0:
    aggregate_file = frames[0]
    previous_frame = None
    idx = 0
    for frame in frames:
      if idx % 15 == 0:
        prepared_frame = frame
        #prepared_frame = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        #prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5,5), sigmaX=0)
        if (previous_frame is None):
          # First frame; there is no previous one yet
          previous_frame = prepared_frame
          continue

        # diff_frame = np.zeros((1080, 1920, 4), dtype=np.uint8)
        # diff_frame = aggregate_file.copy()
        # calculate difference and update previous frame
        aggregate_file = cv2.absdiff(aggregate_file, prepared_frame)
        # cv2.absdiff(previous_frame, prepared_frame, diff_frame)
        # mask = cv2.cvtColor(diff_frame, cv2.COLOR_BGR2GRAY)
        previous_frame = prepared_frame
      idx += 1
      # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
      # kernel = np.ones((5, 5))
      # diff_frame = cv2.dilate(diff_frame, kernel, 1)
      #aggregate_file = cv2.addWeighted(aggregate_file, 0.5, diff_frame, 0.5, 0)
      # cv2.imwrite(os.path.join('/content/', '111.jpg'), diff_frame)
    resize = cv2.resize(aggregate_file, (224, 224))
    cv2.imwrite(os.path.join('/content/train_images/' + str(int(action)), 't1_' + str(int(frame_id))+'.jpg'), resize)

  # if len(frames) > 0:
  #   aggregate_file = frames[0]
  #   previous_frame = None
  #   for i in range(1, len(frames)-1):
  #     if i % 2 == 0:
        
  #       prepared_frame = frames[i]
  #       #prepared_frame = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
  #       #prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5,5), sigmaX=0)
  #       if (previous_frame is None):
  #         # First frame; there is no previous one yet
  #         previous_frame = prepared_frame
  #         continue
  
  #       diff_frame = aggregate_file.copy()
  #       # calculate difference and update previous frame
  #       #diff_frame = cv2.absdiff(previous_frame, prepared_frame, diff_frame)
  #       cv2.absdiff(previous_frame, prepared_frame, diff_frame)
  #       previous_frame = prepared_frame

  #       # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
  #       # kernel = np.ones((5, 5))
  #       # diff_frame = cv2.dilate(diff_frame, kernel, 1)
  #       aggregate_file = cv2.addWeighted(aggregate_file, 0.5, diff_frame, 0.5, 0)

  #     cv2.imwrite(os.path.join('/content/' + str(int(frame_id)), str(i) + '.jpg'), frames[i])
  #   cv2.imwrite(os.path.join('/content', str(frame_id)+'.jpg'), aggregate_file)

In [ ]:
files = os.listdir('/content/5804')

In [ ]:
files = sorted(files, key= lambda file_name: int(file_name.replace('.jpg', '')))

In [ ]:
def merge_images2(files):
  if len(files) > 0:

    aggregate_file = cv2.imread(os.path.join('/content/5804/', files[0]))
    previous_frame = None
    idx = 0
    for file_name in files:
      if idx % 15 == 0:
        frame = cv2.imread(os.path.join('/content/5804/', file_name))

        prepared_frame = frame
        #prepared_frame = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        #prepared_frame = cv2.GaussianBlur(src=prepared_frame, ksize=(5,5), sigmaX=0)
        if (previous_frame is None):
          # First frame; there is no previous one yet
          previous_frame = prepared_frame
          continue

        # diff_frame = np.zeros((1080, 1920, 4), dtype=np.uint8)
        # diff_frame = aggregate_file.copy()
        # calculate difference and update previous frame
        aggregate_file = cv2.absdiff(aggregate_file, prepared_frame)
        # cv2.absdiff(previous_frame, prepared_frame, diff_frame)
        # mask = cv2.cvtColor(diff_frame, cv2.COLOR_BGR2GRAY)
        previous_frame = prepared_frame
      idx += 1
      # 4. Dilute the image a bit to make differences more seeable; more suitable for contour detection
      # kernel = np.ones((5, 5))
      # diff_frame = cv2.dilate(diff_frame, kernel, 1)
      #aggregate_file = cv2.addWeighted(aggregate_file, 0.5, diff_frame, 0.5, 0)
      # cv2.imwrite(os.path.join('/content/', '111.jpg'), diff_frame)
    cv2.imwrite(os.path.join('/content/', '111.jpg'), aggregate_file)

In [ ]:
merge_images2(files)

In [ ]:
!mkdir 5804

mkdir: cannot create directory ‘5804’: File exists


In [ ]:
df_test = pd.read_csv('/content/test/fix_test.csv')

In [ ]:
all_frames = df_test['frame']
all_ids = df_test['id']

In [ ]:
current_idx = 0
current_frame_id = all_frames[current_idx]
current_id = all_ids[current_idx]
frames = []
employee_predict = []

In [ ]:
cap = cv2.VideoCapture(os.path.join('/content/test/', 'test.avi'))
while True:
# for i in range(10000):
  ret, frame = cap.read()
  frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
  #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
  if ret == False:
    break

  #cv2.imwrite(os.path.join('/content', 'frame_crop_'+str(frame_id)+'.jpg'), frame)
  if frame_id % 1000 == 0:
    print(frame_id)

  if (len(frames) > 300):
    frames = frames[-150:]
  frames.append(frame)

  if (current_id == 1190.0):
    cv2.imwrite(os.path.join('/content/', '111.jpg'), frame)

  need_detect = frame_id > current_frame_id + 25 * 2
  if need_detect:
    #employee_predict.append(detect_people_object_test(frames[-100:], current_frame_id, current_id))
    #merge_images(frames[-150:], current_frame_id)

    current_idx += 1
    current_frame_id = all_frames[current_idx]
    current_id = all_ids[current_idx]

    
    while current_frame_id == 0:
      employee_predict.append(0)
      current_idx += 1
      current_frame_id = all_frames[current_idx]
      current_id = all_ids[current_idx]
  # if int(current_frame_id) == int(frame_id) or int(current_frame_id) - 25 == int(frame_id) or int(current_frame_id)-50 == int(frame_id) or int(current_frame_id)+25 == int(frame_id):
  #   

  # cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  # cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
  # current_frame += 1
  #cap.set(cv2.CAP_PROP_POS_MSEC, seconds)
  #seconds += 1
  
cap.release()

1000.0
2000.0
3000.0
4000.0
5000.0
6000.0
7000.0
8000.0
9000.0
10000.0
11000.0
12000.0
13000.0
14000.0
15000.0
16000.0
17000.0
18000.0
19000.0
20000.0
21000.0
22000.0
23000.0
24000.0
25000.0
26000.0
27000.0
28000.0
29000.0
30000.0
31000.0
32000.0
33000.0
34000.0
35000.0
36000.0
37000.0
38000.0
39000.0
40000.0
41000.0
42000.0
43000.0
44000.0
45000.0
46000.0
47000.0
48000.0
49000.0
50000.0
51000.0
52000.0
53000.0
54000.0
55000.0
56000.0
57000.0
58000.0
59000.0
60000.0
61000.0
62000.0
63000.0
64000.0
65000.0
66000.0
67000.0
68000.0
69000.0
70000.0
71000.0
72000.0
73000.0
74000.0
75000.0
76000.0
77000.0
78000.0
79000.0
80000.0
81000.0
82000.0
83000.0
84000.0
85000.0
86000.0
87000.0
88000.0
89000.0
90000.0
91000.0
92000.0
93000.0
94000.0
95000.0
96000.0
97000.0
98000.0
99000.0
100000.0
101000.0
102000.0
103000.0
104000.0
105000.0
106000.0
107000.0
108000.0
109000.0
110000.0
111000.0
112000.0
113000.0
114000.0
115000.0
116000.0
117000.0
118000.0
119000.0
120000.0
121000.0
122000.0
123000.0
1

KeyError: ignored

In [ ]:
cap = cv2.VideoCapture(os.path.join('/content/test/', 'test.avi'))
# while True:
for i in range(10000):
  ret, frame = cap.read()
  frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
  #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
  if ret == False:
    break

  #cv2.imwrite(os.path.join('/content', 'frame_crop_'+str(frame_id)+'.jpg'), frame)
  if frame_id % 1000 == 0:
    print(frame_id)

  if (len(frames) > 300):
    frames = frames[-150:]
  frames.append(frame)
    
  need_detect = frame_id > current_frame_id + 25 * 2
  if need_detect:
    employee_predict.append(detect_people_object_test(frames[-100:], current_frame_id, current_id))
    #merge_images(frames[-150:], current_frame_id)

    current_idx += 1
    current_frame_id = all_frames[current_idx]
    current_id = all_ids[current_idx]
    while current_frame_id == 0:
      employee_predict.append(0)
      current_idx += 1
      current_frame_id = all_frames[current_idx]
      current_id = all_ids[current_idx]
  # if int(current_frame_id) == int(frame_id) or int(current_frame_id) - 25 == int(frame_id) or int(current_frame_id)-50 == int(frame_id) or int(current_frame_id)+25 == int(frame_id):
  #   

  # cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  # cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
  # current_frame += 1
  #cap.set(cv2.CAP_PROP_POS_MSEC, seconds)
  #seconds += 1
  
cap.release()

In [ ]:
!mkdir train_images

In [ ]:
!mkdir train_images/1
!mkdir train_images/2
!mkdir train_images/3
!mkdir train_images/4
!mkdir train_images/5
!mkdir train_images/6
!mkdir train_images/7
!mkdir train_images/8
!mkdir train_images/9
!mkdir train_images/10
!mkdir train_images/11
!mkdir train_images/12
!mkdir train_images/13
!mkdir train_images/14
!mkdir train_images/15
!mkdir train_images/16
!mkdir train_images/17
!mkdir train_images/18
!mkdir train_images/19

In [ ]:
!mkdir train_images/1

In [ ]:
# [2, 5, 7, 0, 0, 2, 7, 2, 2, 0]

In [ ]:
test_files= os.listdir('/content/test_images')

In [ ]:
df_train = pd.read_csv('/content/train/fix_train1.csv')

In [ ]:
all_frames = df_train['frame']
all_ids = df_train['id']
all_action = df_train['action']

In [ ]:
current_idx = 0
current_frame_id = all_frames[current_idx]
current_id = all_ids[current_idx]
current_action = all_action[current_idx]
frames = []
employee_predict = []

In [ ]:
cap = cv2.VideoCapture(os.path.join('/content/train/', 'train1.avi'))
while True:
# for i in range(10000):
  ret, frame = cap.read()
  frame_id = cap.get(cv2.CAP_PROP_POS_FRAMES)
  #print(cap.get(cv2.CAP_PROP_POS_FRAMES))
  if ret == False:
    break

  #cv2.imwrite(os.path.join('/content', 'frame_crop_'+str(frame_id)+'.jpg'), frame)
  if frame_id % 1000 == 0:
    print(frame_id)

  if (len(frames) > 300):
    frames = frames[-150:]
  frames.append(frame)
    
  need_detect = frame_id > current_frame_id + 25 * 2
  if need_detect:
    #employee_predict.append(detect_people_object_test(frames[-100:], current_frame_id, current_id))
    merge_images(frames[-150:], current_frame_id, current_action)

    current_idx += 1
    current_frame_id = all_frames[current_idx]
    current_id = all_ids[current_idx]
    current_action = all_action[current_idx]
    while current_frame_id == 0:
      # employee_predict.append(0)
      current_idx += 1
      current_frame_id = all_frames[current_idx]
      current_id = all_ids[current_idx]
      current_action = all_action[current_idx]
  # if int(current_frame_id) == int(frame_id) or int(current_frame_id) - 25 == int(frame_id) or int(current_frame_id)-50 == int(frame_id) or int(current_frame_id)+25 == int(frame_id):
  #   

  # cv2.imwrite(os.path.join('/content/frames/', 'frame_'+str(frame_idx)+'.jpg'), frame)
  # cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
  # current_frame += 1
  #cap.set(cv2.CAP_PROP_POS_MSEC, seconds)
  #seconds += 1
  
cap.release()

1000.0
2000.0
3000.0
4000.0
5000.0
6000.0
7000.0
8000.0
9000.0
10000.0
11000.0
12000.0
13000.0
14000.0
15000.0
16000.0
17000.0
18000.0
19000.0
20000.0
21000.0
22000.0
23000.0
24000.0
25000.0
26000.0
27000.0
28000.0
29000.0
30000.0
31000.0
32000.0
33000.0
34000.0
35000.0
36000.0
37000.0
38000.0
39000.0
40000.0
41000.0
42000.0
43000.0
44000.0
45000.0
46000.0
47000.0
48000.0
49000.0
50000.0
51000.0
52000.0
53000.0
54000.0
55000.0
56000.0
57000.0
58000.0
59000.0
60000.0
61000.0
62000.0
63000.0
64000.0
65000.0
66000.0
67000.0
68000.0
69000.0
70000.0
71000.0
72000.0
73000.0
74000.0
75000.0
76000.0
77000.0
78000.0
79000.0
80000.0
81000.0
82000.0
83000.0
84000.0
85000.0
86000.0
87000.0
88000.0
89000.0
90000.0
91000.0
92000.0
93000.0
94000.0
95000.0
96000.0
97000.0
98000.0
99000.0
100000.0
101000.0
102000.0
103000.0
104000.0
105000.0
106000.0
107000.0
108000.0
109000.0
110000.0
111000.0
112000.0
113000.0
114000.0
115000.0
116000.0
117000.0
118000.0
119000.0
120000.0
121000.0
122000.0
123000.0
1

KeyError: ignored

In [ ]:
new_employee = []

In [ ]:
employee_predict[180:185]

[4, 7, 2, 0, 0]

In [ ]:
for i, row in df_test.iterrows():
  if row['frame'] != 0:
    if str(row['id'])+'.jpg' not in test_files:
      print(i, row['id'])
  


4 139
12 147
13 148
14 149
36 171
41 176
42 177
67 202
183 318
184 319
231 366
232 367
324 459
325 460
377 512
420 555
422 557
424 559
425 560
426 561
427 562
428 563
446 581
447 582
503 638
508 643
567 702
576 711
583 718
585 720
587 722
593 728
599 734
607 742
633 768
634 769
641 776
723 858
867 1002
879 1014
960 1095
1011 1146
1013 1148
1014 1149
1093 1228
1094 1229
1117 1252
1118 1253


In [ ]:
employee_predict.insert(1120, 0)

In [ ]:
len(employee_predict)

1121

In [ ]:
df_solution = pd.read_csv('/content/simple_solution.csv')

In [ ]:
df_solution['employee'] = employee_predict

In [ ]:
df_solution.to_csv('solution.csv', index=False)

In [ ]:
df_solution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id        1121 non-null   int64
 1   employee  1121 non-null   int64
 2   action    1121 non-null   int64
dtypes: int64(3)
memory usage: 26.4 KB


In [ ]:
df_solution

,id,employee,action
0,135,0,15
1,136,0,4
2,137,0,3
3,138,0,13
4,139,0,13
...,...,...,...
1116,1251,0,14
1117,1252,0,14
1118,1253,0,14
1119,1254,0,14


Классификация